此notebook用于检测获取数据的准确性

In [1]:
from qmtbt import QMTStore
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt

xtquant241014.1.2已经发布,前往 http://dict.thinktrader.net/nativeApi/download_xtquant.html 查看更新说明


In [2]:
from xtquant.xtdata import download_history_data2
#coding=utf-8

import pandas as pd
import sys
sys.path.append(r'E:\\software\\QMT\\userdata_mini')

# 加载xtquant库
from xtquant.xtdata import *

def on_progress(data):
    '''补充历史数据回调函数'''
    print(data) 
    
if __name__ == "__main__":
    print("xtdata demo")
    # 根据stock_list下载数据
    stock_list = ['603909.SH','300450.SZ','600740.SH','002362.SZ','600750.SH']
    field_list = ['time','open','close','low','high','volume'] # 提取的字段
    download_history_data2(stock_list, period='1d', start_time='20230201', end_time='20230223', callback=on_progress)
    print('download_history_data2 finished')
    
    # 获取股票close数据
    ret = get_market_data(field_list, stock_list, period='1d', start_time='', end_time='', count=5, dividend_type='front', fill_data=True)
    print(ret['close'].T)


xtdata demo
download_history_data2 finished
{'finished': 1, 'total': 5, 'stockcode': '', 'message': '603909.SH'}
          603909.SH  300450.SZ  600740.SH  002362.SZ  600750.SH
20241122       9.15     22.627       4.48      14.96      21.57
20241125       9.39     22.627       4.48      14.96      21.57
20241126       9.27     22.627       4.48      14.96      21.57
20241127       9.34     22.627       4.48      14.96      21.57
20241128       9.44     22.627       4.48      14.96      21.57


In [4]:
import backtrader as bt
import pandas as pd

# 定义一个简单的策略，用于记录每条 K 线的数据
class PrintData(bt.Strategy):
    def __init__(self):
        pass

    def next(self):
        # 获取当前的 K 线数据
        current_data = self.datas[0]
        
        # 提取当前 K 线的数据
        date = current_data.datetime.date(0)  # 获取日期
        open_price = current_data.open[0]     # 获取开盘价
        high_price = current_data.high[0]     # 获取最高价
        low_price = current_data.low[0]       # 获取最低价
        close_price = current_data.close[0]   # 获取收盘价
        volume = current_data.volume[0]       # 获取成交量
        
        # 输出数据
        print(f"Date: {date}, Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

# 初始化 cerebro 引擎
cerebro = bt.Cerebro()

# 加载数据
store = QMTStore()
data = store.getdata(dataname='000001.SZ', timeframe=bt.TimeFrame.Days, fromdate=datetime(2020, 1, 1),
                      todate=datetime(2021, 1, 1), live=False,dividend_type='none', fill_data=False)

# 将数据添加到 cerebro
cerebro.adddata(data)

# 添加策略
cerebro.addstrategy(PrintData)

# 运行回测
cerebro.run()
cerebro.plot()


下载数据000001.SZ
000001.SZ历史数据装载成功！
1
Date: 2020-01-02, Open: 14.774000000000001, High: 15.073999999999998, Low: 14.674000000000001, Close: 14.994, Volume: 1530231.0
Date: 2020-01-03, Open: 15.064, High: 15.434000000000001, Low: 15.044, Close: 15.304000000000002, Volume: 1116194.0
Date: 2020-01-06, Open: 15.134, High: 15.463999999999999, Low: 15.033999999999999, Close: 15.193999999999999, Volume: 862083.0
Date: 2020-01-07, Open: 15.253999999999998, High: 15.403999999999996, Low: 15.073999999999998, Close: 15.273999999999997, Volume: 728607.0
Date: 2020-01-08, Open: 15.123999999999999, High: 15.174, Low: 14.753999999999998, Close: 14.783999999999999, Volume: 847824.0
Date: 2020-01-09, Open: 14.934000000000001, High: 15.053999999999998, Low: 14.654000000000002, Close: 14.914000000000001, Volume: 1031636.0
Date: 2020-01-10, Open: 14.913999999999998, High: 14.933999999999997, Low: 14.644, Close: 14.814, Volume: 585548.0
Date: 2020-01-13, Open: 14.873999999999999, High: 15.154, Low: 14.734, Cl

<IPython.core.display.Javascript object>

C:\Users\21009\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\backends\backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


[[<Figure size 640x480 with 4 Axes>]]

In [5]:

def get_k_data(stock_code, begin: datetime, end: datetime):
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d"))
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = ['name', 'code', 'date', 'open', 'close', 'high', 'low', 'volume', 'turnover']
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(['name', 'code', "date"], axis=1, inplace=True)
    return k_dataframe

start_time = datetime(2020, 1, 1)
end_time = datetime(2021, 1, 1)
cerebro = bt.Cerebro()

dataframe = get_k_data('000001', begin=start_time, end=end_time)
data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)

# 将数据添加到cerebro引擎
cerebro.adddata(data)

# 运行策略（这里没有添加策略，只是为了展示绘图功能）
cerebro.run()

# 绘制图表
cerebro.plot()


<IPython.core.display.Javascript object>

C:\Users\21009\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\backends\backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


[[<Figure size 640x480 with 4 Axes>]]

In [2]:
from xtquant import xtdata

# 方法1：直接获取所有板块列表（推荐）
sector_list = xtdata.get_sector_list()
print("所有板块列表：", sector_list)

***** xtdata连接成功 *****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: E:\software\QMT\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

所有板块列表： ['上期所', '上证A股', '上证B股', '上证期权', '上证转债', '中金所', '创业板', '大商所', '沪市ETF', '沪市债券', '沪市基金', '沪市指数', '沪深A股', '沪深B股', '沪深ETF', '沪深债券', '沪深基金', '沪深指数', '沪深转债', '深市ETF', '深市债券', '深市基金', '深市指数', '深证A股', '深证B股', '深证期权', '深证转债', '科创板', '科创板CDR', '能源中心', '连续合约', '郑商所', '香港联交所指数', '香港联交所股票', 'CSRC1农锟斤拷锟街★拷锟斤拷锟斤拷锟斤拷业', 'CSRC1水锟斤拷锟斤拷锟斤拷锟斤拷锟酵癸拷锟斤拷锟斤拷施锟斤拷锟斤拷业', 'CSRC1住锟睫和诧拷锟斤拷业', 'CSRC1锟侥伙拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷业', 'CSRC1锟缴匡拷业', 'CSRC1锟桔猴拷', 'CSRC1锟斤拷通锟斤拷锟戒、锟街达拷锟斤拷锟斤拷锟斤拷业', 'CSRC1��Ϣ���䡢�������Ϣ��������ҵ', 'CSRC1锟斤拷学锟叫撅拷锟酵硷拷锟斤拷锟斤拷锟斤拷业', 'CSRC1锟斤拷锟截诧拷业', 'CSRC1���\u07ba��������ҵ', 'CSRC1锟斤拷锟斤拷', 'CSRC1锟斤拷锟斤拷业', 'CSRC1��������Ṥ��', 'CSRC1锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷业', 'CSRC1锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷燃锟斤拷锟斤拷水锟斤拷锟斤拷锟酵癸拷应业', 'CSRC1锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷业', 'CSRC2煤炭锟斤拷锟缴猴拷洗选业', 'CSRC2木锟侥加癸拷锟斤拷木锟斤拷锟斤拷锟劫★拷锟截★拷锟斤拷锟斤拷品业', 'CSRC2农业', 'CSRC2农锟斤拷食品锟接癸拷

用来测试添加多只股票

In [6]:
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt
from qmtbt import QMTStore
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt
plt.style.use('default')
from sko.GA import GA

import backtrader as bt
from qmtbt import QMTStore
from datetime import datetime
from xtquant import xtdata, xtconstant
import math
import backtrader as bt
from strategies import TestStrategy, AnotherStrategy ,my_broker
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
import optuna
from strategies import *




class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


class MultiSMACrossStrategy(bt.Strategy):
    params = (
        ('fast_length', 5),
        ('slow_length', 25)
    )
     
    def __init__(self):
        self.crossovers = []
         
        for d in self.datas: 
            ma_fast = bt.ind.SMA(d, period = self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period = self.params.slow_length)
 
            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))
 
    def next(self):
        for i, d in enumerate(self.datas):
            if not self.getposition(d).size:
                if self.crossovers[i] > 0: 
                    self.buy(data = d, size=100)
            elif self.crossovers[i] < 0: 
                if self.getposition(d).size > 0:
                    self.close(data = d)


if __name__ == '__main__':
    # 获取数据
    start_time = datetime(2020, 1, 1)
    end_time = datetime(2021, 1, 1)
    cerebro = bt.Cerebro()
    
    # dataframe = get_k_data('000001', begin=start_time, end=end_time)
    # data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)
    store = QMTStore()
    
    code_list = xtdata.get_stock_list_in_sector('上证A股')

    datas = store.getdatas(code_list=code_list[:10], timeframe=bt.TimeFrame.Days, fromdate=datetime(2024, 1, 1),
    todate=datetime(2025, 1, 1),live=False)

    for data in datas:
        cerebro.adddata(data)
    cerebro.addstrategy(TestStrategy)
    start_cash = 1000000
    cerebro.broker.setcash(start_cash)
    cerebro.broker.setcommission(commission=0.00025)
    cerebro.run()
    cerebro.plot(iplot=False)

    # print(cerebro.runstrats[0])
    print(cerebro.broker.getvalue())

下载数据600051.SH
600051.SH历史数据装载成功！
下载数据605090.SH
605090.SH历史数据装载成功！
下载数据600025.SH
600025.SH历史数据装载成功！
下载数据601222.SH
601222.SH历史数据装载成功！
下载数据688031.SH
688031.SH历史数据装载成功！
下载数据603335.SH
603335.SH历史数据装载成功！
下载数据688045.SH
688045.SH历史数据装载成功！
下载数据603341.SH
603341.SH历史数据装载成功！
下载数据600967.SH
600967.SH历史数据装载成功！
下载数据603237.SH
603237.SH历史数据装载成功！
2024-03-01, Close, 5.64
2024-03-04, Close, 5.57
2024-03-05, Close, 5.45
模拟买入，股票代码: 600051.SH, 价格: 5.45, 数量: 200
2024-03-05, BUY CREATE, 5.45
2024-03-06, BUY EXECUTED, 5.41
2024-03-06, Close, 5.52
2024-03-07, Close, 5.51
2024-03-08, Close, 5.52
2024-03-11, Close, 5.65
2024-03-12, Close, 5.76
2024-03-13, Close, 5.70
2024-03-13, SELL CREATE, 5.70
模拟卖出，股票代码: 600051.SH, 价格: 5.70, 数量: 200
2024-03-14, SELL EXECUTED, 5.70
2024-03-14, Close, 5.74
2024-03-15, Close, 5.79
2024-03-18, Close, 5.95
2024-03-19, Close, 5.93
2024-03-20, Close, 5.99
2024-03-21, Close, 6.02
2024-03-22, Close, 5.91
2024-03-25, Close, 5.81
模拟买入，股票代码: 600051.SH, 价格: 5.81, 数量: 200
2024-03-25, BUY CREA

KeyboardInterrupt: 